In [ ]:
import re
from typing import Optional

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.scenario.scenario import Tag
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.ConversionAnalyzer.SpotAnalyzer import SpotAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter
from OpenSCENARIO2CR.Osc2CrConverterResult import Osc2CrConverterResult
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.PPSBuilder import PPSBuilder

In [ ]:
scenario_path = "/path/to/esmini/resources/xosc/acc-test.xosc"
run_viewer = False
render_to_gif = False
draw_plots = True

In [ ]:
# Setup EsminiWrapper
esmini_wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
esmini_wrapper.min_time = 13
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False
esmini_wrapper.random_seed = 0

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = None
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)
    raise Exception

In [ ]:
if render_to_gif:
    esmini_wrapper.render_scenario_to_gif(scenario_path, "scenario.gif")

In [ ]:
pps_builder = PPSBuilder()
pps_builder.time_interval = AbsRel.relative(Interval(-10, 0))
pps_builder.position_length_factor = AbsRel.absolute(50)
pps_builder.position_width_factor = AbsRel.absolute(10)
pps_builder.position_rotation = AbsRel.relative(0)
pps_builder.position_center_x = AbsRel.relative(0)
pps_builder.position_center_y = AbsRel.relative(0)
pps_builder.velocity_interval = AbsRel.relative(Interval(-5, 5))
pps_builder.orientation_interval = None

In [ ]:
converter = Osc2CrConverter(
    author="ADD AUTHOR HERE",
    affiliation="ADD AFFILIATION HERE",
    source="ADD SOURCE HERE",
    tags={Tag.SIMULATED},
)

converter.sim_wrapper = esmini_wrapper
converter.pps_builder = pps_builder

converter.dt_cr = 0.1
converter.keep_ego_vehicle = True
converter.trim_scenario = False
converter.use_implicit_odr_file = True
converter.analyzers = {
    EAnalyzer.SPOT: SpotAnalyzer(stride=1),
    EAnalyzer.DRIVABILITY: None,  # Will be initialized with default parameters
    EAnalyzer.STL: None,  # Will be initialized with default parameters
}
# If you only want to run with default parameters for analyzers you can also use:
# converter.analyzers = [EAnalyzer.SPOT_ANALYZER, EAnalyzer.DRIVABILITY_CHECKER, EAnalyzer.STL_MONITOR]

converter.dt_sim = 0.01
converter.odr_file_override = None
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE)

In [ ]:
result = converter.run_conversion(scenario_path)

In [ ]:
assert isinstance(result, Osc2CrConverterResult)

scenario = result.scenario
pps = result.planning_problem_set
analysis = result.analysis
stats = result.statistics

In [ ]:
def draw_obstacle_in_color(renderer, obstacle, color, trajectory_color: Optional = None, time: int = 0):
    assert renderer is not None
    assert obstacle is not None
    assert color is not None
    renderer.draw_dynamic_obstacle(
        obstacle,
        call_stack=(),
        draw_params={
            "time_begin": time,
            "trajectory": {
                "draw_trajectory": trajectory_color is not None,
                "draw_continuous": True,
                "line_width": 1.5,
                "facecolor": "#000000" if trajectory_color is None else trajectory_color,
            },
            "dynamic_obstacle": {
                "draw_shape": True,
                "vehicle_shape": {
                    "occupancy": {
                        "shape": {
                            "rectangle": {
                                "facecolor": color,
                                "edgecolor": "black",
                            },
                        }
                    },
                },
            },
        }
    )

In [ ]:
if draw_plots:
    lim_y = 10
    lim_x = lim_y * 3
    x_offset = 15
    x_size = 5
    y_size = 4

    highlight_color = "#0065BD"
    highlight_color_darker = "#005293"

    normal_color = "#E37222"
    normal_color_darker = "#994d17"

    time_steps_with_spot = [45, 75]
    time_steps_no_spot = [60]

    rnd = MPRenderer(figsize=(2 * x_size, 0.7 * y_size), plot_limits=[-10, 260, -30, 30], )
    scenario.draw(rnd, draw_params={
        "trajectory": {"draw_trajectory": False},
        "dynamic_obstacle": {"draw_shape": False,},
        "time_begin": 0
    })
    draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[0], highlight_color, highlight_color_darker, time=0)
    for i in range(1, len(scenario.dynamic_obstacles)):
        draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[i], "#E37222", "#994d17", time=0)
    pps.draw(rnd)
    rnd.render()
    plt.tight_layout()
    plt.savefig("overview.png")
    plt.show()

    for t in time_steps_no_spot:
        rnd = MPRenderer(
            figsize=(x_size, 0.5 * y_size),
            draw_params={"focus_obstacle_id": scenario.dynamic_obstacles[0].obstacle_id},
            plot_limits=[-lim_x + x_offset, lim_x + x_offset, -lim_y, lim_y],
        )
        scenario.draw(rnd, draw_params={
            "trajectory": {"draw_trajectory": False},
            "dynamic_obstacle": {"draw_shape": False,},
            "time_begin": t
        })
        draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[0], highlight_color, highlight_color_darker, time=t)
        for i in range(1, len(scenario.dynamic_obstacles)):
            draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[i], "#E37222", "#994d17", time=t)
        pps.draw(rnd)
        rnd.render()
        plt.tight_layout()
        plt.savefig(f"step-{t}.png")
        plt.show()
    for t in time_steps_with_spot:
        fig, axs = plt.subplots(2, 1, tight_layout=True, figsize=(x_size, y_size))
        assert isinstance(fig, plt.Figure)

        rnd = MPRenderer(
            ax=axs[0],
            draw_params={"focus_obstacle_id": scenario.dynamic_obstacles[0].obstacle_id},
            plot_limits=[-lim_x + x_offset, lim_x + x_offset, -lim_y, lim_y],
        )
        scenario.draw(rnd, draw_params={
            "trajectory": {"draw_trajectory": False},
            "dynamic_obstacle": {"draw_shape": False,},
            "time_begin": t
        })
        draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[0], highlight_color, highlight_color_darker, time=t)
        for i in range(1, len(scenario.dynamic_obstacles)):
            draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[i], "#E37222", "#994d17", time=t)
        pps.draw(rnd)
        rnd.render()

        res = analysis[EAnalyzer.SPOT][1][stats.ego_vehicle]
        restore = {}
        for o_id, o_prediction in res.predictions[t].items():
            if o_id == scenario.dynamic_obstacles[0].obstacle_id:
                continue
            restore[o_id] = scenario.obstacle_by_id(o_id).prediction
            scenario.obstacle_by_id(o_id).prediction = o_prediction

        try:
            rnd = MPRenderer(
                ax=axs[1],
                draw_params={"focus_obstacle_id": scenario.dynamic_obstacles[0].obstacle_id},
                plot_limits=[-lim_x + x_offset, lim_x + x_offset, -lim_y, lim_y],
            )
            scenario.draw(rnd, draw_params={
                "trajectory": {"draw_trajectory": False},
                "dynamic_obstacle": {"draw_shape": False,},
                "time_begin": t
            })
            draw_obstacle_in_color(rnd, scenario.dynamic_obstacles[0], highlight_color, highlight_color_darker, time=t)
            for o_id, o_prediction in restore.items():
                scenario.obstacle_by_id(o_id).prediction = o_prediction
                draw_obstacle_in_color(rnd, scenario.obstacle_by_id(o_id), "#E37222", None, time=t)

            pps.draw(rnd)
            rnd.render()

            fig.savefig(f"step-{t}.png")
            fig.show()
        finally:
            for o_id, o_prediction in restore.items():
                scenario.obstacle_by_id(o_id).prediction = o_prediction